In [1]:
import os
os.chdir('..') # move to the root directory (from dev)

In [2]:
import yaml
import warnings

from prob_models import prior_model, posterior_model
from energy_model import EnergyModel
from utils.gurobi_env import get_Gurobi_WLS_env
from utils.plotting import init_profile_fig, add_profile

/Users/mal84/opt/miniconda3/envs/EP-VOI/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Load settings and params
with open(os.path.join('configs','base_settings.yaml'), 'r') as f: settings = yaml.safe_load(f)
model_settings,prob_settings = [settings[key] for key in ['model_settings','probability_settings']]

with open(os.path.join('configs','base_params.yaml'), 'r') as f: params = yaml.safe_load(f)
base_cost_dict,base_ts_dict,base_storage_dict = [params[key] for key in ['cost_values','timeseries_values','storage_values']]

In [4]:
n_samples = 5

In [5]:
# Sample scenarios from prior
theta_scenarios, z_scenarios = prior_model(prob_settings, base_cost_dict, base_ts_dict, base_storage_dict, n_samples=n_samples)

for m,scenario in enumerate(theta_scenarios):
    scenario.to_file(os.path.join('test_expt','samples','thetas',f'scenario_{m}.yaml'))
for m,scenario in enumerate(z_scenarios):
    scenario.to_file(os.path.join('test_expt','samples','zs',f'scenario_{m}.yaml'))

In [6]:
# Solve prior SP
model = EnergyModel()
with warnings.catch_warnings():
    # filter pandas warnings, `DeprecationWarning: np.find_common_type is deprecated.`
    warnings.simplefilter("ignore", category=FutureWarning)

    for scenario in theta_scenarios:
        scenario.load_timeseries(os.path.join(*settings['dataset_dir']))
    model.generate_SP(theta_scenarios,model_settings)

    env = get_Gurobi_WLS_env(silence=False)
    model.solve(env=env)

    _,_,_ = model.save_results(os.path.join('test_expt','prior','results.yaml'))
    model.save_scenarios(os.path.join('test_expt','prior','scenarios'))

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2512934
Academic license 2512934 - for non-commercial use only - registered to ma___@cam.ac.uk


Writing binary variables.: 100%|██████████| 1/1 [00:00<00:00, 1773.49it/s]


Read LP format model from file /private/var/folders/xj/y66v8msx1h52bvwjcrhl3ws40000gp/T/linopy-problem-j3e0bi6t.lp
Reading time = 1.52 seconds
obj: 1051211 rows, 613210 columns, 3586223 nonzeros
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (mac64[arm] - Darwin 23.6.0 23H124)

CPU model: Apple M2 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Academic license 2512934 - for non-commercial use only - registered to ma___@cam.ac.uk
Optimize a model with 1051211 rows, 613210 columns and 3586223 nonzeros
Model fingerprint: 0x701ccf79
Variable types: 613206 continuous, 4 integer (4 binary)
Coefficient statistics:
  Matrix range     [1e-03, 1e+09]
  Objective range  [2e-04, 1e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 5e+08]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve added 0 rows and 43784 columns
Presolve removed 131417 rows and 0 columns
Presolve time: 3.73

Dual values of MILP couldn't be parsed


In [7]:
fig = init_profile_fig(y_titles={'y1':'Energy flow (kWh)', 'y2':'State of Charge (kWh)', 'y3':'Price ($/kWh)'})

fig = add_profile(fig, model.grid_energies[0].solution.values, name='Grid load')
fig = add_profile(fig, model.scenarios[0].norm_wind_gen*model.model.variables.wind_capacity.solution.values, name=f'Wind generation')
fig = add_profile(fig, model.scenarios[0].norm_solar_gen*model.model.variables.solar_capacity.solution.values, name=f'Solar generation')
fig = add_profile(fig, model.scenarios[0].elec_prices, name='Electricity price', yaxis='y3', visible='legendonly')
for i,tech in enumerate(model.techs):
    if model.model.variables[f'{tech}_capacity'].solution > 0:
        fig = add_profile(fig, getattr(model.model.variables,f'SOC_i{i}_s0').solution, name=f'{tech} SoC', yaxis=f'y2', visible='legendonly')

fig.write_html(f'temp_plot.html')
fig.show()

In [8]:
# Solve posterior SPs
for m,scenario in enumerate(z_scenarios):

    # sample from posterior
    vartheta_scenarios = posterior_model(scenario, prob_settings, n_samples=n_samples)

    model = EnergyModel()
    with warnings.catch_warnings():
        # filter pandas warnings, `DeprecationWarning: np.find_common_type is deprecated.`
        warnings.simplefilter("ignore", category=FutureWarning)

        for scenario in vartheta_scenarios:
            scenario.load_timeseries(os.path.join(*settings['dataset_dir']))
        model.generate_SP(vartheta_scenarios,model_settings)

        env = get_Gurobi_WLS_env(silence=True)
        model.solve(env=env)

        _,_,_ = model.save_results(os.path.join('test_expt','posterior',f'scenario_{m}','results.yaml'))
        model.save_scenarios(os.path.join('test_expt','posterior',f'scenario_{m}','scenarios'))

Writing binary variables.: 100%|██████████| 1/1 [00:00<00:00, 1703.62it/s]
Dual values of MILP couldn't be parsed
Writing binary variables.: 100%|██████████| 1/1 [00:00<00:00, 1271.00it/s]
Dual values of MILP couldn't be parsed


Writing binary variables.: 100%|██████████| 1/1 [00:00<00:00, 1786.33it/s]
Dual values of MILP couldn't be parsed
Writing binary variables.: 100%|██████████| 1/1 [00:00<00:00, 1691.93it/s]
Dual values of MILP couldn't be parsed
Writing binary variables.: 100%|██████████| 1/1 [00:00<00:00, 1552.30it/s]
Dual values of MILP couldn't be parsed
